<a href="https://colab.research.google.com/github/PeerChristensen/invoiceParsing/blob/main/FSC_certificeringer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FSC-certificeringer fra fakturaer i .edi- og .xml-formater

In [41]:
!pip uninstall pandas
!pip install pandas==1.1.5
#!pip3 install --upgrade pandas
from bs4 import BeautifulSoup as bs4
import pandas as pd
import re
import os

path = "files"
file_names = os.listdir(path)
files = [os.path.join(path,i) for i in file_names]
for i in files:
    print(i)

Found existing installation: pandas 1.3.4
Uninstalling pandas-1.3.4:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/pandas-1.3.4.dist-info/*
    /usr/local/lib/python3.7/dist-packages/pandas/*
Proceed (y/n)? y
  Successfully uninstalled pandas-1.3.4
     |████████████████████████████████| 9.5 MB 5.4 MB/s 


files/Staedtler Nordic AS_20210519_040021930_26.edi
files/Esselte_04501_INOUT_INVOIC.21711_20210825120730853.edi
files/Burde invoice_000759106_2021-08-11-10-45-31{c732b041-f0d5-483e-b83b-3b92722a3b59}.xml


## Top-level funktion

In [42]:
def parse_all_files(files: list) -> pd.DataFrame:
    """
    Top-level function to parse both .edi and .xml files.
    Returns a dataframe with key data from input files.
    """
    edi_files = [i for i in files if i.endswith('.edi')]
    xml_files = [i for i in files if i.endswith('.xml')]

    list_of_dfs = []
    if len(edi_files) > 0:
        edi_data = parse_edi_files(edi_files)
        list_of_dfs.append(edi_data)
    if len(xml_files) > 0:
        xml_data = parse_xml_files(xml_files)
        list_of_dfs.append(xml_data)

    stacked_df = pd.concat(list_of_dfs)
    return stacked_df

## Funktioner til processering af .edi-filer






In [43]:
def get_edi_content(file: str) -> list:

    content = open(file, "r", encoding="latin1").read()
    content_list = content.split("'")
    return content_list


def get_creditor_gln(content: list) -> str:
    """
    Get Creditor GLN from edifact file
    Q: Does the relevant element always start with "UNB+UNOC:3+
    Q: Is this the same as Creditor number"
    """
    for i in content:
        if i.startswith('UNB+UNOC'):
            creditor_gln__string = i
            break
    creditor_gln = re.search('UNB\\+UNOC:3\\+(.*?)\\:', creditor_gln__string).group(1)
    return creditor_gln


def get_invoice_no(content: list) -> str:
    """Get invoice number from edifact file
    """
    for i in content:
        if i.startswith('BGM+380+'):
            invoice_string = i
            break
    invoice_no = re.search('BGM\\+380\\+(.*?)\\+9', invoice_string).group(1)
    return invoice_no


def get_barcodes(content: list) -> list:

    barcode_lines = [i for i in content if i.startswith("LIN")]
    barcodes = []
    for line in barcode_lines:
        barcode = re.search('\\+\\+(.*?)\\:', line).group(1)
        barcodes.append(barcode)
    return barcodes


def get_name_and_fsc(content: list) -> list:

    name_and_fsc_lines = [i for i in content if i.startswith("IMD+F+")]
    name_and_fsc_list = []
    for line in name_and_fsc_lines:
        name_and_fsc = re.search(':::(.*)', line).group(1)
        name_and_fsc_list.append(name_and_fsc)
    return name_and_fsc_list


def make_df_from_edi_file(file: str) -> pd.DataFrame:
    """
    This function calls various helper functions to create a dataframe after parsing
    the relevant content from .edi files.
    """
    content = get_edi_content(file)            # get content
    cred_gln_no = get_creditor_gln(content)    # get creditor GLN number
    inv_no = get_invoice_no(content)           # get invoice number
    barcodes = get_barcodes(content)           # get barcodes
    names_and_fsc = get_name_and_fsc(content)  # get names and FSC codes

    df = pd.DataFrame({"CreditorGLN": [cred_gln_no],
                       "InvoiceNum": [inv_no],
                       "Barcode": [barcodes],
                       "ProductNameFSC": [names_and_fsc]})

    df = df.apply(pd.Series.explode).reset_index(drop=True)
    #df = df.explode(["Barcode", "ProductNameFSC"]).reset_index(drop=True)
    return df


def parse_edi_files(files: list) -> pd.DataFrame:

    list_of_dfs = []
    for file in files:
        df = make_df_from_edi_file(file)
        list_of_dfs.append(df)

    stacked_df = pd.concat(list_of_dfs)
    return stacked_df

## Funktioner til processering af .xml-filer


In [44]:
def get_xml_content(file: str) -> bs4:

    content = open(file, "r", encoding="utf8").read()
    soup = bs4(content, "xml")
    return soup


def get_creditor_gln_xml(soup: bs4) -> int:
    """Get Creditor GLN from xml file"""
    acp_tag = soup.find("cac:AccountingCustomerParty")
    creditor_gln = acp_tag.PartyIdentification.ID.text
    return creditor_gln


def get_invoice_no_xml(soup: bs4) -> str:
    """Get invoice number from xml file"""
    invoice_no = soup.find("cbc:ID").text
    return invoice_no


def get_items_xml(soup: bs4) -> bs4:
    items = soup.findAll("cac:Item")
    return items


def get_barcodes_xml(items: bs4) -> list:
    barcodes = []
    for i in items:
        try:
            barcodes.append(i.ID.text)
        except:
            barcodes.append("")
    return barcodes


def get_name_and_fsc_xml(items: bs4) -> list:
    items_descriptions = []
    for i in items:
        try:
            items_descriptions.append(i.Description.text)
        except:
            items_descriptions.append("")
    return items_descriptions


def make_df_from_xml_file(file: str) -> pd.DataFrame:
    """
    This function calls various helper functions to create a dataframe after parsing
    the relevant content from .xml files.
    """
    content = get_xml_content(file)                # get content
    cred_gln_no = get_creditor_gln_xml(content)    # get creditor GLN number
    inv_no = get_invoice_no_xml(content)           # get invoice number
    items = get_items_xml(content)                 # get items
    barcodes = get_barcodes_xml(items)           # get barcodes
    names_and_fsc = get_name_and_fsc_xml(items)  # get names and FSC codes

    df = pd.DataFrame({"CreditorGLN": [cred_gln_no],
                       "InvoiceNum": [inv_no],
                       "Barcode": [barcodes],
                       "ProductNameFSC": [names_and_fsc]})

    df = df.apply(pd.Series.explode).reset_index(drop=True)
    #df = df.explode(["Barcode", "ProductNameFSC"]).reset_index(drop=True)
    return df


def parse_xml_files(files: list) -> pd.DataFrame:

    list_of_dfs = []
    for file in files:
        df = make_df_from_xml_file(file)
        list_of_dfs.append(df)

    stacked_df = pd.concat(list_of_dfs)
    return stacked_df

## Output i tabelformat

In [45]:
df = parse_all_files(files)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

display(df)

,CreditorGLN,InvoiceNum,Barcode,ProductNameFSC
0,5790001329389,85073973,4007817078242,"LUMOCOLOR PERM F 0,6MM 8 ASS/BOKS"
1,5790001329389,85073973,4007817077856,STAEDTLER METALLIC PEN 2?+1/BK
2,5790001329389,85073973,4007817304679,TEXTSURFER CLASSIC INKJET GUL
3,5790001329389,85073973,4007817049617,TEXTSURFER CLASSIC PASTEL LIMEGRØN
4,5790001329389,85073973,4007817049655,TEXTSURFER CLASSIC VINTAGE AZURBL
5,5790001329389,85073973,4007817304686,TEXTSURFER CLASSIC INKJET ROSA
6,5790001329389,85073973,4007817049778,TEXTSURFER CLASSIC HIDELIGHTER SORT
7,5790001329389,85073973,4007817015636,DISP. VISKEGUMMI MARS PLASTIC 40STK
8,5790001329389,85073973,4007817341063,LUMOCOLOR COMPACT WB-MARKER 1-2MM S
9,5790001329389,85073973,4007817328835,LUMOCOLOR WB-MARKER 2MM GRØN
